In [1]:
import time

import bernielib as bl

In [2]:
bl.listSerialPorts()

['COM7', 'COM18', 'COM20']

In [3]:
ber = bl.robot(cartesian_port_name='COM18', pipette_port_name='COM7', misc_port_name='COM20')

In [4]:
ber.robotHome()
ber.pipetteHome()

In [22]:
beads = bl.createSample('eppendorf', 'beads', ber.samples_rack, 0, 0, 800)

In [7]:
beads.getVolume()

800

In [8]:
mix_scenario = {
    50:{
        0: {'X': 0, 'Y': 0},
    },
    100: {
        0: {'X': 2, 'Y': 0},
        1: {'X': 1.5, 'Y': 1.5},
        2: {'X': 0, 'Y': 2},
        3: {'X': -1.5, 'Y': 1.5},
        4: {'X': -2, 'Y': 0},
        5: {'X': -1.5, 'Y': -1.5},
        6: {'X': 0, 'Y': -2},
        7: {'X': 1.5, 'Y': -1.5},
        8: {'X': 0, 'Y': 0},
    },
    200: {
        0: {'X': 3, 'Y': 0},
        1: {'X': 2, 'Y': 2},
        2: {'X': 0, 'Y': 3},
        3: {'X': -2, 'Y': 2},
        4: {'X': -3, 'Y': 0},
        5: {'X': -2, 'Y': -2},
        6: {'X': 0, 'Y': -3},
        7: {'X': 2, 'Y': -2},
        8: {'X': 0, 'Y': 0},
    },
}

In [5]:
ber.pickUpNextTip()

In [18]:
ber.move(z=0)

In [11]:
ber.mix(sample=beads, scenario=mix_scenario, times=3)

In [12]:
samples_list = bl.createSamplesToPurifyList(ber, [50, 100, 200])

In [13]:
samples_list[0].getVolume()

50

In [15]:
ber.pipetteMove(1)

# Calculating how many beads to add

In [16]:
cutoff_list = [1000, 700, 200]
v_beads_list = []
for sample, size in zip(samples_list, cutoff_list):
    v_beads = ber.calcBeadsVol(sample, size)
    v_beads_list.append(v_beads)

In [17]:
v_beads_list

[25.758674747999997, 53.77361189020407, 218.74755015999997]

# Adding beads

In [20]:
def addBeads(robot, sample, beads, v_beads, mix_scenario):
    robot.move(z=0)
    robot.pickUpNextTip()
    robot.move(z=0)
    robot.transferLiquid(beads, sample, v_beads)
    robot.mix(sample, scenario=mix_scenario)
    robot.move(z=0)
    robot.dumpTipToWaste()
    robot.move(z=0)

In [24]:
ber.pickUpNextTip()

In [25]:
ber.transferLiquid(beads, samples_list[0], 50)

In [31]:
ber.move(z=0)

In [32]:
ber.dumpTipToWaste()

In [27]:
addBeads(ber, samples_list[0], beads, v_beads_list[0], mix_scenario)

In [23]:
ber.dumpTipToWaste()

# Mixing procedure (continued)

In [39]:
def mix4(robot, sample, scenario, times=3, v_uptake=200, mix_delay=0.5):
    v_in = sample.getVolume()
    v_max = sample.stype.getMaxVolume()
    z_out = sample.calcAbsLiquidLevelFromVol(v_max, added_length=robot._calcExtraLength())
    
    if sample._isLowVolumeUptakeNeeded(v_uptake):
        # Case when there is low liquid volume
        v_uptake_projected = sample.stype.getCloseToBottomVol()
        z_mix_projected = sample.calcAbsLiquidLevelFromVol(v_uptake_projected, added_length=robot._calcExtraLength())
        if not sample._settingPresent('tube_bottom_z'):
            # Probing the bottom
            robot.moveToSample(sample, z=z_mix_projected)
            z_tube_bottom = robot.moveDownUntilPress(step=0.2, threshold=200)
            sample.setZBottom(z_tube_bottom)
            sample.setBottomTouched()
            # Moving back up after probing
            robot.moveToSample(sample, z=z_out)
            
        z_mix = sample.getZBottom() - 1
        
        # Making sure robot will not uptake bubbles
        if v_uptake > v_in * 0.8:
            v_uptake = v_in * 0.8
            
        robot.movePipetteToVolume(v_uptake)
        
        robot.moveToSample(sample, z=z_mix)
        
        robot._pipetteUpAndDownInPlace(delay=mix_delay, times=times)
        robot.uptakeLiquid(sample, v_uptake)
        
        
        
        robot.moveToSample(sample, z=z_out)
        
        robot.pipetteMove(40)
        time.sleep(mix_delay)
        robot.pipetteMove(0)

        
        

In [42]:
beads = bl.createSample('eppendorf', 'beads', ber.samples_rack, 0, 0, 75)

In [41]:
beads.purge()

In [11]:
ber.pickUpNextTip()

In [25]:
ber.move(z=0)

In [53]:
mix4(ber, beads, scenario=mix_scenario)

## Mixing colours

In [11]:
blue = bl.createSample('eppendorf', 'blue', ber.samples_rack, 0, 0, 1000)
yellow = bl.createSample('eppendorf', 'yellow', ber.samples_rack, 1, 0, 50)

In [50]:
ber.transferLiquid(blue, yellow, 50)

In [56]:
mix4(ber, yellow, scenario=mix_scenario)

In [76]:
def mix5(robot, sample, times=3, v_uptake=200, mix_delay=0.5):
    v_in = sample.getVolume()
    v_max = sample.stype.getMaxVolume()
    z_just_above_liquid = sample.calcAbsLiquidLevelFromVol(v_in*1.2, added_length=robot._calcExtraLength())
    z_max = sample.calcAbsLiquidLevelFromVol(v_max, added_length=robot._calcExtraLength())
    
    if v_uptake > v_in * 0.8:
        v_uptake = v_in * 0.8
        
    times = times - 1
    robot.uptakeLiquid(sample, v_uptake)
    z_uptake = robot.getPosition(axis='Z')
    for i in range(times):
        robot.moveToSample(sample, z=z_just_above_liquid)
        robot.dispenseLiquid(sample, v_uptake, plunger_retract=False, move_up_after=False, in_place=True)
        robot.moveToSample(sample, z=z_uptake)
        robot.uptakeLiquid(sample, v_uptake, in_place=True)
    
    robot.moveToSample(sample, z=z_max)
    robot.pipetteMove(40)
    time.sleep(mix_delay)
    robot.touchWall(sample)
    robot.pipetteMove(0)
    sample.setVolume(v_in)

In [77]:
mix5(ber, yellow, times=3)

In [10]:
yellow.getVolume()

50

In [10]:
yellow.purge()

In [16]:
yellow._settingPresent('tube_bottom_z')

False

In [19]:
ber._probeTubeZBottom(yellow)

142.43

In [24]:
ber.move(z=0)

In [23]:
ber._getTubeZBottom(yellow)

142.43

# Mixing according to the script

In [5]:
import pandas as pd

In [92]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')

In [93]:
script

,Height,H_relative_to,Plunger_pos,dx,dy,min_vol_condition,max_vol_condition,delay,comment
0,0,top,down,0,0,0,1700,0.5,Initially lowering the plunger
1,1,bottom,up,0,0,0,250,0.5,mixing in place
2,1,bottom,down,0,0,0,250,0.5,mixing in place
3,1,bottom,up,0,0,0,250,0.5,mixing in place
4,1,bottom,down,0,0,0,250,0.5,mixing in place
5,1,bottom,up,0,0,0,250,0.5,mixing in place
6,1,bottom,down,0,0,0,250,0.5,mixing in place
7,1,bottom,up,0,0,0,1700,0.5,"uptaking from bottom, releasing to the top"
8,5,surface,down,0,0,0,1700,0.5,"uptaking from bottom, releasing to the top"
9,1,bottom,up,0,0,0,1700,0.5,"uptaking from bottom, releasing to the top"


In [8]:
def mixByScript(robot, sample, script, vol_uptake_fraction=0.8):
    v_in = sample.getVolume()
    v_max = sample.stype.getMaxVolume()
    z_liquid = sample.calcAbsLiquidLevelFromVol(v_in, added_length=robot._calcExtraLength())
    z_max = sample.calcAbsLiquidLevelFromVol(v_max, added_length=robot._calcExtraLength())
    
    number_of_steps = script.shape[0]
    for step in range(number_of_steps):
        
        print ("Executing step ", step)
        
        step_params = script.loc[step]
        # Parameters
        h = step_params.Height
        h_relative_to = step_params.H_relative_to
        plunger_position = step_params.Plunger_pos
        dx = step_params.dx
        dy = step_params.dy
        condition_v_min = step_params.min_vol_condition
        condition_v_max = step_params.max_vol_condition
        d = step_params.delay
        
        # Do I need to skip this step entirely?
        if v_in > condition_v_min and v_in < condition_v_max:
            # perform step
        
            # Figuring out absolute Z of the operation. If necessary, will probe the tube bottom.
            if h_relative_to == 'top':
                z = z_max + h # Getting deeper into the tube compared to the tube top. Positive -> deeper
            elif h_relative_to == 'bottom':
                z_bottom = robot._getTubeZBottom(sample)
                z = z_bottom - h # Above the absolute Z of the tube.
            elif h_relative_to == 'surface':
                z = z_liquid - h # Above the surface level
            else:
                print ("Please provide valid H_relative_to value. Valid values are top, bottom, surface.")
            
            # Identifying uptake volume
            v_uptake = v_in * vol_uptake_fraction
            if v_uptake > 200:
                v_uptake = 200
            
            if plunger_position == 'up':
                p = 0
            elif plunger_position == 'down':
                p = robot.calcPipettePositionFromVolume(v_uptake)
            else:
                print ("Please provide valid Plunger_pos value. Valid values are up and down.")
        
            # Moving to the position for the operation
            robot.moveToSample(sample, z=z, z_hop=0)
            robot.moveAxisDelta('X', dx)
            robot.moveAxisDelta('Y', dy)
        
            # Performing pipette operation (actual mixing)
            robot.pipetteMove(p)
            time.sleep(d)
            
            # Moving back x and y:
            robot.moveAxisDelta('X', -dx)
            robot.moveAxisDelta('Y', -dy)
            
            # Now getting to the next step.
            
        
        
        #print (h, h_relative_to, plunger_position, dx, dy, v_min, v_max, d)
    
    #Purging at the end of the protocol
    robot.moveToSample(sample, z=z_max, z_hop=0)
    robot.pipetteMove(40)
    time.sleep(d)
    robot.touchWall(sample)
    robot.pipetteMove(0)

In [53]:
yellow = bl.createSample('eppendorf', 'yellow', ber.samples_rack, 1, 0, 600)

In [10]:
ber.pickUpNextTip()

In [52]:
ber.move(z=0)

In [126]:
mixByScript(ber, yellow, script)

Executing step  0
Executing step  1
Executing step  2
Executing step  3
Executing step  4
Executing step  5
Executing step  6
Executing step  7
Executing step  8
Executing step  9
Executing step  10
Executing step  11
Executing step  12
Executing step  13
Executing step  14
Executing step  15
Executing step  16
Executing step  17
Executing step  18
Executing step  19
Executing step  20
Executing step  21
Executing step  22
Executing step  23
Executing step  24
Executing step  25
Executing step  26
Executing step  27
Executing step  28
Executing step  29
Executing step  30
Executing step  31
Executing step  32
Executing step  33
Executing step  34
Executing step  35
Executing step  36


In [127]:
ber.move(z=0)

In [66]:
ber.moveMagnetsTowardsTube()

In [67]:
ber.moveMagnetsAway()

# Settings to be entered just once

In [5]:
ber.setBeadsVolumeCoef(a=4.99325349e-01, b=-9.91043764e+00, c=2.57585836e+04)

In [6]:
ber.getBeadsVolumeCoef()

(0.499325349, -9.91043764, 25758.5836)

In [7]:
test_type = bl.sample_type('eppendorf')

In [8]:
test_sample = bl.sample('test_sample', test_type)

In [14]:
test_sample.setVolume(50)

In [16]:
ber.calcBeadsVol(test_sample, 1000)

25.758674747999997

In [43]:
ber.pipetteServoUp()

In [42]:
ber.pipetteServoDown()

In [24]:
ber.calibrateRack(rack='tips')

(75.5, 150.57, 168.3)

In [5]:
ber.move(z=180)

In [6]:
ber.move(z=0)

In [111]:
ber.writeAndWaitCartesian("M503")

'echo:  G21    ; Units in mm\n\necho:Filament settings: Disabled\necho:  M200 D3.00\necho:  M200 D0\necho:Steps per unit:\necho:  M92 X10.00 Y10.00 Z100.00 E500.00\necho:Maximum feedrates (units/s):\necho:  M203 X2000.00 Y2000.00 Z4000.00 E25.00\necho:Maximum Acceleration (units/s2):\necho:  M201 X10000 Y10000 Z100 E10000\necho:Acceleration (units/s2): P<print_accel> R<retract_accel> T<travel_accel>\necho:  M204 P250.00 R10000.00 T250.00\necho:Advanced: S<min_feedrate> T<min_travel_feedrate> B<min_segment_time_us> X<max_xy_jerk> Z<max_z_jerk> E<max_e_jerk>\necho:  M205 S0.00 T0.00 B20000 X2.00 Y2.00 Z1.00 E5.00\necho:Home offset:\necho:  M206 X0.00 Y0.00 Z0.00\necho:PID settings:\necho:  M301 P22.20 I1.08 D114.00\necho:Z-Probe Offset (mm):\necho:  M851 Z0.00\nok\n'

In [128]:
ber.writeAndWaitCartesian("M201 Z200")

'ok\n'